In [ ]:
# imports
from src.lab import read_chord_file
from src.audio import convert_chromagram_to_dataframe, get_chromagram_from_file
from src.utils import annotate_chord_sequence

from src.hmm import (
    build_transition_probability_matrix,
    compute_initial_state_probabilities,
    filter_and_normalize_initial_probabilities,
    build_gaussian_hmm,
)

In [2]:
DEST_FOLDER = 'lab_and_audio_files'
let_it_be_intro_path = 'sounds/Let it Be Intro.wav'

lab_file_path = f'{DEST_FOLDER}/Let_It_Be.lab'
chords_annotation = read_chord_file(lab_file_path)
chords_annotation.head()

,start,end,chord
0,0.000000,0.175157,C
1,0.175157,1.852358,C
2,1.852358,3.454535,G
3,3.454535,4.720022,A:min
4,4.720022,5.126371,A:min


In [3]:
raw_chromagram = convert_chromagram_to_dataframe(*get_chromagram_from_file(let_it_be_intro_path))
chromagram = annotate_chord_sequence(raw_chromagram, chords_annotation)
chromagram.head()

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,start,end,chord
0,0.328111,0.392393,0.430106,0.430240,0.171732,0.105401,0.109222,0.133969,0.166579,0.214087,0.308256,0.359433,0.000000,0.046419,<START>
1,0.664177,0.352761,0.109753,0.110626,0.366218,0.163517,0.087432,0.182613,0.090595,0.068443,0.157639,0.412985,0.046419,0.092837,C
2,0.754787,0.184479,0.045408,0.124241,0.429377,0.168346,0.106199,0.206016,0.082103,0.026816,0.040539,0.321305,0.092837,0.139256,C
3,0.734501,0.146537,0.047722,0.111363,0.507049,0.167623,0.137559,0.243638,0.065114,0.015817,0.024087,0.236238,0.139256,0.185674,C
4,0.833918,0.198937,0.056337,0.078762,0.421175,0.114144,0.089200,0.182769,0.042916,0.006769,0.013399,0.147586,0.185674,0.232093,C


In [4]:
transition_matrix = build_transition_probability_matrix(chromagram)
transition_matrix

next_chord,<END>,A:min,C,F,G,<START>
current_chord,,,,,,
<START>,0.00,0.000000,1.000000,0.000000,0.000000,0.0
A:min,0.00,0.972222,0.000000,0.027778,0.000000,0.0
C,0.01,0.000000,0.970000,0.000000,0.020000,0.0
F,0.00,0.000000,0.028169,0.971831,0.000000,0.0
G,0.00,0.014493,0.000000,0.014493,0.971014,0.0
<END>,1.00,0.000000,0.000000,0.000000,0.000000,0.0


In [5]:
initial_state_probs = compute_initial_state_probabilities()
display(initial_state_probs[:5])

initial_state_probs = filter_and_normalize_initial_probabilities(initial_state_probs, transition_matrix)
display(initial_state_probs)

A     0.22
Ab    0.04
B     0.02
C     0.18
C#    0.04
dtype: float64

next_chord
<END>      0.746269
A:min      0.000000
C          0.134328
F          0.014925
G          0.104478
<START>    0.000000
dtype: float64

In [ ]:
mu_array, states_cov_matrices = get_mu_sigma_from_chroma(pcp)
h_markov_model = build_gaussian_hmm(initial_state_probs, transition_matrix, mu_array, states_cov_matrices)